In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
#         print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler as std
from sklearn.decomposition import PCA
import os
import gc
import sys
import seaborn as sns
import torch 
import torch.nn as nn
import keras
import datetime
from tqdm import tqdm 
import gc 
import pickle as p
from tensorflow.keras import initializers

In [3]:
def the_model():
    x_input =keras.layers.Input(shape=(cfg.num_rows, cfg.num_cols))
    x1 = keras.layers.Bidirectional(keras.layers.LSTM(units=256, return_sequences=True))(x_input)
    x2 = keras.layers.Bidirectional(keras.layers.LSTM(units=128, return_sequences=True))(x1)
    z1 = keras.layers.Bidirectional(keras.layers.GRU(units=64, return_sequences=True))(x1)
    c = keras.layers.Concatenate(axis=2)([x2, z1])
#     x3 = keras.layers.Bidirectional(keras.layers.LSTM(units=256, return_sequences=True))(c)
#     x30=keras.layers.Bidirectional(keras.layers.LSTM(units=256,return_sequences=True))(x1)
#     c1=keras.layers.Concatenate(axis=2)([x3,x30])
    x4 = keras.layers.GlobalMaxPooling1D()(c)#(c1)
    #x41=keras.layers.Dense(units=32,activation="relu")(x4)
    x411=keras.layers.Dense(units=16,activation="relu", kernel_initializer=initializers.RandomNormal(stddev=0.01),
    bias_initializer=initializers.Zeros())(x4)
    x42=keras.layers.Dropout(0.05)(x411)
    x_output = keras.layers.Dense(1, activation='sigmoid')(x42)
    model = keras.models.Model(inputs=x_input, outputs=x_output, name='lstm_model')
    return model

In [10]:
model = keras.models.load_model(".")

In [ ]:
df=pd.read_csv("/kaggle/input/cleaned-weld-right-data/Test_data.csv")

In [ ]:
df.Production.unique(),df["Order Operation No"].unique()
le = preprocessing.LabelEncoder()
col=["Production","Order Operation No"]

df[col[0]]=df[col[0]].values.astype(np.str_)
df[col[1]]=df[col[1]].values.astype(np.str_)
df[col[0]] = le.fit_transform(df[col[0]].values)
df[col[1]] = le.fit_transform(df[col[1]].values)
df.info()

In [ ]:
"""
domain specific features
"""
def feature1(df):
    #df["Voltage"]=df["Voltage"]
    df["Power"]=(df["Current"])*((df["Voltage"])+1)
    df["Delta_Temperature"]= df["Job Temp"]-df["Temperature"]
    df["Resistance"]=(df["Voltage"]+1)/(df["Current"])
    df["Delta_Charge"]=(df["Current"])*(1/1000)*df["Delta_Time"]
    df["Alpha"]=df["Humidity"]/df["Delta_Temperature"]
    df["Heat"]=df["Power"]*df["Delta_Time"]*(1/1000)
    return df
"""
Lag features
"""
def lag(df):
    coll=["Power","Delta_Time","Delta_Temperature","Alpha","Heat",'Current','Humidity','Flow','Job Temp']
    for i in coll:
        temp=[]
        for j in range(len(df[i])):
            if j==0 :
                temp.append(0)
            else:
                temp.append(df[i].iloc[j] - df[i].iloc[j-1] )
        df[f"lag_{i}"]=temp
    return df
""")
pca , ica 
"""
def pca_originals(df):
    """
     to make our ml model stable
        
    """
    names=coll=["Current","Humidity","Temperature","Flow","Job Temp","Voltage"]
    pca= PCA(n_components=None)
    #df[coll]=std_transform(df[coll],coll)
    df[["Pca_"+i for i in coll]]= pca.fit_transform(df[names])
    return df

In [ ]:
    def time_feature(df):
        """
        gives time in milli second
        1st = time
        2nd = delta_time
        hr : min : sec : m_sec
        
        """
        a=df.Time
        b=df.Date
        a1=[]
        a2=[]
        current=0
        for i in tqdm(range(len(df))):
            if current == b[i]:
                t=a[i].split(":")
            elif i == 0 or current!=b[i]:
                current=b[i]
                t=[0,0,0,0]
            a1.append(int(t[0])*60*60*1000+int(t[1])*60*1000+int(t[2])*1000+int(t[3]))# hr min sec mili sec
            if i == 0:
                a2.append(0)
            else:
                a2.append(a1[i]-a1[i-1])
        return a1,a2
    
    def remove_feature(df,names):
        """
        remove 1_to_1 features
        
        """
        cols= df.columns
        for i in names:
            del df[i]
        return df
    
    def std_transform(df,names):
        """
        standardize values 
        (before pca)
        
        """
        scale=std()
        df[names]=scale.fit_transform(df[names])
        return df
    
    def divi(df,name):
        """
        type - dictionary 
        keys - unique entries in column
        (to be used with employee code "specifically")
        
        """
        df_code= df.groupby(name)
        df1= {}
        
        for i in df_code:
            df1[i[0]] = i[1]
        return df1
    
    def label(df):
        defs=df.Defect.values
        defs=defs!="No Defect"
        defs=defs*1
        df.Defect=defs
        return df
        
    def prep_training(df, history = 60):
        snips1=[]
        for i in tqdm(range( len(df)) ):
            if df.iloc[i].Defect == 1 and i >= history:
                temp=df.iloc[i-history+1:i+1]
                temp=remove_feature(temp,names=cols[:1])
                temp=remove_feature(temp,names=["Defect","label"])
                snips1.append(temp)
        snips0=[]
        """
        startification of entries
        """
        for i in tqdm(range( len(df)) ):
            if df.iloc[i].Defect == 0 and i >= history and len(snips0)<len(snips1)*2:
                temp=df.iloc[i-history+1:i+1]
                temp=remove_feature(temp,names=cols[:1])
                temp=remove_feature(temp,names=["Defect","label"])
                snips0.append(temp)
        snips=[snips0,snips1]
        return snips

In [ ]:
            
def transform1(df):
    #1st convert time
    (a,b)=time_feature(df)
    df["Time"]=a
    df["Delta_Time"]=b
    df=feature1(df)
    df=lag(df)
    #df=pca_originals(df)
    print(df.head())
    print(1)
    #2nd remove useless cols-> machine / date /  
    col=["Machine","Date"]
    df=remove_feature(df,col)
    print(2)
    #3rd correct labels
    df=label(df)
    print(3)
    #4th 9 workers 9 dfs
    df=divi(df,name=df.columns[0])
    keys=df.keys()
    """
    output={int->id : grouped df}
    """
    print(4)
    #5th std all dfs
    for i in tqdm(df):
        df[i]=prep_training(df[i])

    print(5)
    gc.collect()
    return df
"""
first approach 40 ki history with subtle gradient for transition from 0 to the first 1 
then adapt the curve
"""
    
def cols_update(df):
    return [i for i in df.columns]
dff=df
cols=cols_update(dff)
dft=transform1(dff)

In [ ]:
data=dft

In [ ]:
for i in tqdm(data):
    typee=np.float
#     print(s[0],type(data[i][0][0]))
#     print(data[0][0][0].shape)
    temp0=np.array(data[i][0]).astype(typee)
    temp1=np.array(data[i][1]).astype(typee)
    ln_1=len(data[i][1])
    ln_0=(ln_1)
    ln_0=int(ln_0)
    x=np.concatenate((temp0[:ln_1],temp1))
    data[i]=[x,y]

In [ ]:
x=[]
for i in data:
    x.extend(data[i][0])

In [ ]:
preds=model.predict(x)